<font size = "+3"><center><b>The University of British Columbia Okanagan</b></font><br>
Faculty of Applied Science, School of Engineering

<br>
<font size = "+2"><b>ENGR 418</font><br>
Applied Machine Learning for Engineers <br>
Dr. Anas Chaaban <br></b>

<br>
<font size = "+2"><b>Project Stage 1</b><br></font>
October 30, 2023 <br>

<br>
<b>Group Members:</b> <br>
Alif Aiman Ahmad Zukiman | 35338490 <br>
Anas Chellabi | 

In [2]:
# test

import numpy as np

a = [1,2,3]
b = [4,5,6]

c = np.array([a,b]).T
print(c)

# test test test
grav_accel = 9.81


[[1 4]
 [2 5]
 [3 6]]


AttributeError: 'NoneType' object has no attribute 'sdhaiksd'